In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 20:28].values
classes = np.reshape(classes, (2000, 700, 8))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 8)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM, Bidirectional, Masking

In [5]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Dense(8, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [6]:
accuq8 = []
precisionsq8 = []
recallsq8 = []
f1q8 = []

In [8]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 8))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 8))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accuq8.append(balanced_accuracy_score(y_test, predicted)) 
    precisionsq8.append(precision_score(y_test, predicted, average = 'weighted'))
    recallsq8.append(recall_score(y_test, predicted, average = 'weighted'))
    f1q8.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [9]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0819 11:46:05.495461  3000 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0819 11:46:05.501445  3000 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0819 11:46:05.502442  3000 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   0    2    3 ... 1997 1998 1999] TEST: [   1   10   20   29   32   33   37   47   51   53   57   58   69   73
   79   80   83   84   85   86   88   92   99  103  107  109  113  114
  115  121  122  125  127  131  149  151  154  161  166  170  174  182
  184  185  190  202  203  209  220  230  234  243  245  246  250  252
  259  263  279  281  293  307  311  324  330  336  338  342  344  346
  351  357  371  380  382  387  393  401  411  412  415  420  430  434
  436  440  442  450  456  457  459  461  475  484  488  498  499  504
  508  509  519  523  531  536  540  542  544  556  558  559  560  563
  567  571  582  586  592  593  596  604  608  609  619  623  624  636
  637  641  643  645  650  657  658  668  669  677  682  685  693  696
  710  711  717  724  725  728  739  741  750  752  754  760  769  774
  775  778  779  785  788  789  807  808  819  820  821  822  831  832
  833  836  838  845  847  860  872  875  878  886  891  903  910  921
  929  931  935  939  941  9

W0819 11:46:07.913779  3000 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 28s 17ms/sample - loss: 0.6389 - acc: 0.1217
Epoch 2/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6263 - acc: 0.1191
Epoch 3/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6254 - acc: 0.1200
Epoch 4/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6231 - acc: 0.1227
Epoch 5/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6228 - acc: 0.1229
Epoch 6/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6229 - acc: 0.1222
Epoch 7/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6220 - acc: 0.1230
Epoch 8/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6217 - acc: 0.1230
Epoch 9/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6214 - acc: 0.1231
Epoch 10/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 

c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


...

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1429
           1       0.28      0.02      0.04     22202
           2       0.00      0.00      0.00      4321
           3       0.34      0.99      0.51     37225
           4       0.00      0.00      0.00       752
           5       0.70      0.04      0.07     22259
           6       0.00      0.00      0.00      9693
           7       0.00      0.00      0.00     12450

    accuracy                           0.34    110331
   macro avg       0.17      0.13      0.08    110331
weighted avg       0.31      0.34      0.19    110331

Acurácia
0.13046327219173365
Precisao
0.31294135414185104
Recall
0.3442550144564991
F1
0.19310999050294714
[[    0    24     0  1388     0    17     0     0]
 [    0   454     0 21617     0   131     0     0]
 [    0    79     0  4219     0    23     0     0]
 [    0   425     0 36691     0   109     0     0]
 [    0     0     0   752     0     0 

1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6200 - acc: 0.1243
Epoch 50/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6196 - acc: 0.1244
Epoch 51/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6182 - acc: 0.1244
Epoch 52/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6139 - acc: 0.1321
Epoch 53/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6224 - acc: 0.1223
Epoch 54/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6206 - acc: 0.1242
Epoch 55/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6205 - acc: 0.1243
Epoch 56/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6201 - acc: 0.1243
Epoch 57/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.6203 - acc: 0.1243
Epoch 58/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.61

c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1287
           1       0.31      0.02      0.03     21564
           2       0.00      0.00      0.00      3989
           3       0.33      0.99      0.49     33338
           4       0.00      0.00      0.00       695
           5       0.68      0.04      0.08     21115
           6       0.00      0.00      0.00      9228
           7       0.00      0.00      0.00     11622

    accuracy                           0.33    102838
   macro avg       0.16      0.13      0.08    102838
weighted avg       0.31      0.33      0.18    102838

Acurácia
0.13124612678382808
Precisao
0.31103104387349284
Recall
0.333417608277096
F1
0.18329882225951952
[[    0    14     0  1253     0    20     0     0]
 [    0   376     0 20999     0   189     0     0]
 [    0    43     0  3922     0    24     0     0]
 [    0   230     0 33030     0    78     0     0]
 [    0     4     0   690     0     1  

1600/1600 [==============================] - 24s 15ms/sample - loss: 0.3393 - acc: 0.2411
Epoch 50/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.3344 - acc: 0.2429
Epoch 51/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.3307 - acc: 0.2442
Epoch 52/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.3284 - acc: 0.2447
Epoch 53/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.3229 - acc: 0.2469
Epoch 54/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.3205 - acc: 0.2472
Epoch 55/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.3192 - acc: 0.2475
Epoch 56/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.3162 - acc: 0.2489
Epoch 57/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.3136 - acc: 0.2498
Epoch 58/100
1600/1600 [==============================] - 24s 15ms/sample - loss: 0.31

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_9 (Bidirection (None, 700, 200)          97600     
_________________________________________________________________
dropout_9 (Dropout)          (None, 700, 200)          0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_10 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_11 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 700, 200)         

In [1]:
print('Acurácias total')
print(accuq8)
a = np.array(accuq8)
print(a.mean())
print('Precision total')
print(precisionsq8)
p = np.array(precisionsq8)
print(p.mean())
print('Recalls total')
print(recallsq8)
r = np.array(recallsq8)
print(r.mean())
print('F1 total')
print(f1q8)
f = np.array(f1q8)
print(f.mean())

Acurácias total


NameError: name 'accuq8' is not defined